# 一、样本选取、定义

#### 模型目标输出两组Y，可作为企业临时额度参考，或考虑未来加入风控规则中

* Y1：预测未来30天内的订单总金额

* Y2：预测未来60天内的订单总金额

* X：客户基础信息、天眼查信息(样本查询量太少无法作为特征输入)、订单信息、询价信息（指标开发完成后可批量扩大特征维度投喂给模型）

* 预测准确率检验指标：RMSE

#### 样本时间窗口：
* 预测日期设定为：当前日期往前推61天
* 样本选取：这60天期间（不包含当前日期）有成功交易的客户（保证Y1和Y2至少有一个不为0）
* X中订单信息和询价信息的时间窗口设定为：预测日期往前推30、60、90、120、150、180、360天
* X中天眼查的查询时间应取：在预测日期前最近一次记录

#### 样本分层：
* 训练集80%，测试集20%
* 5折交叉验证

# 二、算法选择

回归问题可选算法：

参考Kaggle房价预测比赛：https://blog.csdn.net/wydyttxs/article/details/79680814

1. Elastic-Net Regression
2. Random Forest 
3. SVM
4. XGBoost/LightGBM
5. Ensemble 多个算法组合加权平均

# 三、模型开发

* 数据清洗
* 特征工程
* 模型调参
* 效果评估
* 模型输出

## 1. 数据清洗

#### 先在mysql平台生产库中全部运行data_extraction.sql & data_analysis.sql，清洗底表给python读取

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from urllib import quote_plus as urlquote

pd.set_option('max_columns',1000)
pd.set_option('max_row',300)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


# 开思平台生产库
password = "Casstime@0719"
engine = create_engine('mysql+pymysql://ximu:%s@cassec-demo.mysql.rds.aliyuncs.com:3306/ec_admin'%urlquote(password))

# 开思金融数据库
password2 = "Casstime@com1112"
engine2 = create_engine('mysql+pymysql://reader:%s@172.21.21.78:3306/kscredit'%urlquote(password2))

def get_sql(sql,engine):
    df = pd.read_sql(sql=sql,con = engine)
    print df.shape
    print df.columns
    return df

### 客户

In [2]:
cust = get_sql("select * from xmtemp.sales_forecast_customer",engine)
cust.head()

(79995, 26)
Index([u'id', u'name', u'code', u'group_classfication', u'auth_dur',
       u'crt_dur', u'province', u'is_enabled', u'is_virtual',
       u'is_authenticated', u'authenticated_stamp', u'created_stamp',
       u'province_geo_name', u'city_geo_name', u'county_geo_name',
       u'customer_manager', u'CASSLOAN_credit_limit',
       u'CASSLOAN_credit_limit_max', u'CASSLOAN_credit_st',
       u'CASSLOAN_MAX_DEAD_LINE', u'COMPANY_ACCOUNT_credit_limit',
       u'COMPANY_ACCOUNT_credit_st', u'COMPANY_ACCOUNT_credit_limit_max',
       u'COMPANY_ACCOUNT_MAX_DEAD_LINE', u'legal_person_identity',
       u'business_license_name'],
      dtype='object')


,id,name,code,group_classfication,auth_dur,crt_dur,province,is_enabled,is_virtual,is_authenticated,authenticated_stamp,created_stamp,province_geo_name,city_geo_name,county_geo_name,customer_manager,CASSLOAN_credit_limit,CASSLOAN_credit_limit_max,CASSLOAN_credit_st,CASSLOAN_MAX_DEAD_LINE,COMPANY_ACCOUNT_credit_limit,COMPANY_ACCOUNT_credit_st,COMPANY_ACCOUNT_credit_limit_max,COMPANY_ACCOUNT_MAX_DEAD_LINE,legal_person_identity,business_license_name
0,005cn10dakWDE8jO4Ii,厦门忠煜达贸易有限公司,C0170321,COMPANY,175.00000,175,province3,1,0,1,2019-05-13 11:13:36,2019-05-13 11:09:32,福建省,厦门市,集美区,黄毅超,nan,nan,None,nan,nan,None,nan,nan,4303251215121X,厦门忠煜达贸易有限公司
1,007IOQO2XAgYAb8bSK6,深圳市华晟汽车服务有限公司,C0197985,COMPANY,339.00000,339,province2,1,0,1,2018-11-30 14:39:27,2018-11-30 14:25:16,广东省,深圳市,龙岗区,耿兴福,2000000.00000,2000000.00000,0,25.00000,nan,3,nan,nan,4303251215121X,深圳市华晟汽车服务有限公司
2,007PCU4iRMJjgussRH4,宁波市鄞州下应宏良汽车装饰服务部,C0161666,COMPANY,433.00000,433,province2,1,0,1,2018-08-28 11:26:11,2018-08-28 11:22:37,浙江省,宁波市,鄞州区,王涛,nan,nan,None,nan,nan,None,nan,nan,4303251215121X,宁波市鄞州下应宏良汽车装饰服务部
3,00aFb4RNggymEhPooZt,匠心独运汽车服务,C0127771,COMPANY,nan,-75,province1,1,0,0,NaT,2020-01-18 18:41:31,新疆维吾尔自治区,乌鲁木齐市,新市区,,nan,nan,None,nan,nan,None,nan,nan,None,None
4,00AzGku3HdjhTKhLZWp,佛山市禅城区图盈汽车维修服务中心,C0104913,COMPANY,56.00000,56,province2,1,0,1,2019-09-09 17:49:50,2019-09-09 17:47:02,广东省,佛山市,禅城区,黄民光,nan,nan,None,nan,nan,None,nan,nan,4303251215121X,佛山市禅城区图盈汽车维修服务中心


In [3]:
len(cust)

79995

### 订单

In [ ]:
# todo 
# 是否该拉平均数？采购单非每日连续进来，拉均值可能抹掉信息

In [4]:
sales = get_sql("select * from xmtemp.sales_forecast_order_analysis",engine)
sales.head()

(18079, 29)
Index([u'company_id', u'later_30', u'later_60', u'duration', u'min_date',
       u'max_date', u'ttl_amt', u'avg_mon_amt', u'avg_ord_amt', u'order_cnt',
       u'avg_mon_cnt', u'min_amt', u'max_amt', u'mon_past_30', u'max_past_30',
       u'mon_past_60', u'max_past_60', u'mon_past_90', u'mon_past_120',
       u'mon_past_150', u'mon_past_180', u'mon_past_360', u'ord_past_30',
       u'ord_past_60', u'ord_past_90', u'ord_past_120', u'ord_past_150',
       u'ord_past_180', u'ord_past_360'],
      dtype='object')


,company_id,later_30,later_60,duration,min_date,max_date,ttl_amt,avg_mon_amt,avg_ord_amt,order_cnt,avg_mon_cnt,min_amt,max_amt,mon_past_30,max_past_30,mon_past_60,max_past_60,mon_past_90,mon_past_120,mon_past_150,mon_past_180,mon_past_360,ord_past_30,ord_past_60,ord_past_90,ord_past_120,ord_past_150,ord_past_180,ord_past_360
0,is0MU4l4Y0OsYPJGk5n,370.00000,370.00000,6.33330,2019-04-29,2019-08-23,33011.40000,5212.32632,1500.51818,22.00000,3.47370,100.00000,8299.20000,0.00000,0.00000,0.00000,0.00000,220.00000,908.25000,1541.00000,1335.83333,2750.95000,nan,nan,330.00000,454.12500,513.66667,471.47059,1500.51818
1,lpNCwtPRarHlfXuvrc1,6589.20000,19929.20000,5.06670,2019-06-06,2019-11-03,47548.40000,9384.55263,660.39444,72.00000,14.21050,15.00000,11430.00000,7200.19000,1466.19000,6221.77000,1466.19000,7366.11333,10987.80000,9422.48000,7924.73333,3962.36667,600.01583,414.78467,450.98653,686.73750,663.55493,660.39444,660.39444
2,12418,201863.96000,385372.23000,22.40000,2018-01-02,2019-11-02,5447146.79000,243176.19598,1590.87231,3424.00000,152.85710,8.00000,85000.00000,140146.37000,11007.00000,168398.04500,20330.00000,192599.91333,207541.43750,209150.06800,195874.87167,213925.17083,881.42371,1005.36146,1077.98459,1109.84726,1147.91475,1106.63769,1277.16520
3,2KzewjZjjxdCyHLjxwr,3833.00000,5750.20000,4.86670,2019-06-12,2019-10-27,45058.00000,9258.49315,1325.23529,34.00000,6.98630,30.00000,4225.00000,12730.00000,3800.00000,10707.50000,3800.00000,11719.33333,11170.75000,9011.60000,7509.66667,3754.83333,1414.44444,1259.70588,1171.93333,1354.03030,1325.23529,1325.23529,1325.23529
4,p3cmI6GZwHph5OYqZzB,16857.09000,19409.99000,3.83330,2019-07-13,2019-11-03,26000.77000,6782.80956,962.99148,27.00000,7.04350,85.60000,5056.20000,1752.65000,795.00000,4237.95000,5056.20000,7040.38667,6500.19250,5200.15400,4333.46167,2166.73083,438.16250,847.59000,1111.64000,962.99148,962.99148,962.99148,962.99148


In [5]:
len(sales)

18079


### 询价

In [6]:
# 询价数据量过大，仅选在订单表中有表现的company_id
idt = get_sql('''select a.*
              from xmtemp.sales_forecast_inquiry_analysis a 
              join xmtemp.sales_forecast_order_analysis b  
              on a.garage_company_id = b.company_id
              ''',engine)
idt.head()

(94088, 8)
Index([u'garage_company_id', u'period', u'car_brand', u'inquiry_duration',
       u'inquiry_cnt', u'need_cnt', u'avg_mon_inq_cnt', u'avg_mon_inq_need'],
      dtype='object')


,garage_company_id,period,car_brand,inquiry_duration,inquiry_cnt,need_cnt,avg_mon_inq_cnt,avg_mon_inq_need
0,00DUvW5UNUmiHGVLG6z,past_all,common,78,24,36.00000,9.23080,13.84620
1,00DUvW5UNUmiHGVLG6z,past_all,luxury,1,4,14.00000,120.00000,420.00000
2,00DUvW5UNUmiHGVLG6z,past_all,premium,105,436,1184.00000,124.57140,338.28570
3,00L49e9DHwxryOOoavd,past_all,common,335,8,29.00000,0.71640,2.59700
4,00L49e9DHwxryOOoavd,past_all,premium,353,12,34.00000,1.01980,2.88950


In [7]:
company_id = idt['garage_company_id'].drop_duplicates().reset_index().drop(columns=['index'])

inquiry = pd.DataFrame(columns = ('garage_company_id'
                                  ,'luxury_past_all_inq_dur','luxury_past_all_inq_cnt','luxury_past_all_need_cnt'
                                  ,'premium_past_all_inq_dur','premium_past_all_inq_cnt','premium_past_all_need_cnt'
                                  ,'common_past_all_inq_dur','common_past_all_inq_cnt','common_past_all_need_cnt'
                                  ,'cheap_past_all_inq_dur','cheap_past_all_inq_cnt','cheap_past_all_need_cnt'
                                  
                                  ,'luxury_past_30_inq_dur','luxury_past_30_inq_cnt','luxury_past_30_need_cnt'
                                  ,'premium_past_30_inq_dur','premium_past_30_inq_cnt','premium_past_30_need_cnt'
                                  ,'common_past_30_inq_dur','common_past_30_inq_cnt','common_past_30_need_cnt'
                                  ,'cheap_past_30_inq_dur','cheap_past_30_inq_cnt','cheap_past_30_need_cnt'
                                  
                                  ,'luxury_past_60_inq_dur','luxury_past_60_inq_cnt','luxury_past_60_need_cnt'
                                  ,'premium_past_60_inq_dur','premium_past_60_inq_cnt','premium_past_60_need_cnt'
                                  ,'common_past_60_inq_dur','common_past_60_inq_cnt','common_past_60_need_cnt'
                                  ,'cheap_past_60_inq_dur','cheap_past_60_inq_cnt','cheap_past_60_need_cnt'
                                 ))

inquiry = pd.concat([company_id,inquiry],axis = 1)

# 去除inquiry重复列
Cols = list(inquiry.columns)
for i,item in enumerate(inquiry.columns):
    if item in inquiry.columns[:i]: Cols[i] = "toDROP"
inquiry.columns = Cols
inquiry = inquiry.drop("toDROP",1)
# print inquiry

##-----------------------------------------------------------------------------------------##

print 'inquiry length: '+str(len(inquiry))

# 一维转多维
for p in ['past_all','past_30','past_60']:
    print p
    for b in ['luxury','premium','common','cheap']:
        print b
#         print 'length:' + str(len(dt))
        for c_id in company_id['garage_company_id'].tolist():
            i = inquiry[inquiry.garage_company_id == c_id].index.tolist()[0]  #取索引行
            try:
                j = idt[(idt.period == p) & (idt.car_brand == b) & (idt.garage_company_id == c_id)].index.tolist()[0]  #取索引行
#                 print i,j
                inquiry[b+'_'+p+'_inq_dur'][i] = idt['inquiry_duration'][j]
                inquiry[b+'_'+p+'_inq_cnt'][i] = idt['inquiry_cnt'][j]
                inquiry[b+'_'+p+'_need_cnt'][i] = idt['need_cnt'][j]
            except Exception,e:
#                 print e
                pass

inquiry length: 15329
past_all
luxury
premium
common
cheap
past_30
luxury
premium
common
cheap
past_60
luxury
premium
common
cheap


In [8]:
inquiry[inquiry.garage_company_id == '00DUvW5UNUmiHGVLG6z'].T

,0
garage_company_id,00DUvW5UNUmiHGVLG6z
luxury_past_all_inq_dur,1
luxury_past_all_inq_cnt,4
luxury_past_all_need_cnt,14.00000
premium_past_all_inq_dur,105
premium_past_all_inq_cnt,436
premium_past_all_need_cnt,1184.00000
common_past_all_inq_dur,78
common_past_all_inq_cnt,24
common_past_all_need_cnt,36.00000


In [9]:
inquiry = inquiry.fillna(0)
inquiry.head(5).T

,0,1,2,3,4
garage_company_id,00DUvW5UNUmiHGVLG6z,00L49e9DHwxryOOoavd,014iUAQwjNqfdcFkREm,01nCphol4nioeHzM658,01WTcbp4nBvOiOzSfzl
luxury_past_all_inq_dur,1,0,0,0,344
luxury_past_all_inq_cnt,4,0,0,0,26
luxury_past_all_need_cnt,14.00000,0.00000,0.00000,0.00000,67.00000
premium_past_all_inq_dur,105,353,116,215,469
premium_past_all_inq_cnt,436,12,24,12,440
premium_past_all_need_cnt,1184.00000,34.00000,42.00000,20.00000,1170.00000
common_past_all_inq_dur,78,335,86,80,457
common_past_all_inq_cnt,24,8,16,16,206
common_past_all_need_cnt,36.00000,29.00000,64.00000,34.00000,507.00000


In [10]:
len(inquiry)

15329

### 天眼查（废弃）
历史存量客户有天眼查的记录过少，且未来控制天眼查次数，数据可用度不高

In [110]:
sql_str = '''
SELECT id,cust_code,cust_id,cf_apply_code,cf_apply_id
,json_extract(json_extract(base_info,"$.result"),"$.staffNumRange") AS staffNumRange # 人员规模
,json_extract(json_extract(base_info,"$.result"),"$.fromTime") AS fromTime # 经营开始时间
,json_extract(json_extract(base_info,"$.result"),"$.categoryScore") AS categoryScore # 行业分数
,json_extract(json_extract(base_info,"$.result"),"$.type") AS type # 法人类型，1人2公司
,json_extract(json_extract(base_info,"$.result"),"$.isMicroEnt") AS isMicroEnt # 是否小微企业，0不是1是
,json_extract(json_extract(base_info,"$.result"),"$.percentileScore") AS percentileScore # 企业评分
,json_extract(json_extract(base_info,"$.result"),"$.regNumber") AS regNumber # 注册号
,json_extract(json_extract(base_info,"$.result"),"$.regCapital") AS regCapital # 注册资本
,json_extract(json_extract(base_info,"$.result"),"$.regLocation") AS regLocation # 注册地址
,json_extract(json_extract(base_info,"$.result"),"$.industry") AS industry # 行业
,json_extract(json_extract(base_info,"$.result"),"$.socialStaffNum") AS socialStaffNum # 参保人数
,json_extract(json_extract(base_info,"$.result"),"$.businessScope") AS businessScope # 经营范围
,json_extract(json_extract(base_info,"$.result"),"$.estiblishTime") AS estiblishTime # 成立日期
,json_extract(json_extract(base_info,"$.result"),"$.regStatus") AS regStatus # 企业状态
,json_extract(json_extract(base_info,"$.result"),"$.actualCapital") AS actualCapital # 实收注册资金
,json_extract(json_extract(base_info,"$.result"),"$.companyOrgType") AS companyOrgType # 企业类型
,json_extract(json_extract(base_info,"$.result"),"$.base") AS province # 省份简称
,json_extract(json_extract(holder,"$.result"),"$.total") AS holder_cnt # 股东总数
FROM `outdata_tyanc_data` 
;
'''
tyc = get_sql(sql_str,engine2)
tyc.head(3).T

(1671, 22)
Index([u'cass_code', u'xm_code', u'cust_code', u'cust_id', u'staffNumRange',
       u'fromTime', u'categoryScore', u'TYPE', u'isMicroEnt',
       u'percentileScore', u'regNumber', u'regCapital', u'regLocation',
       u'industry', u'socialStaffNum', u'businessScope', u'estiblishTime',
       u'regStatus', u'actualCapital', u'companyOrgType', u'province',
       u'holder_cnt'],
      dtype='object')


,0,1,2
cass_code,C0124315,C0140846,C0170576
xm_code,XMMEMBER242886933552828416,XMMEMBER242892144593866752,XMMEMBER243090453321224192
cust_code,E242886946739720193,E242892153695506433,E243090465308545025
cust_id,242886946739720192,242892153695506432,243090465308545024
staffNumRange,"""-""",None,"""-"""
fromTime,1493827200000,1358870400000,1501516800000
categoryScore,None,None,None
TYPE,1,1,1
isMicroEnt,1,1,1
percentileScore,5839,4736,5335


In [122]:
ss = pd.merge(pd.merge(sales,cust,left_on = 'company_id',right_on = 'id',how = 'left'),tyc,left_on = 'code',right_on = 'cass_code',how = 'left')
ss.head(5).T

,0,1,2,3,4
company_id,BvhhIisVjEDfgqSPH3G,Kfnw8TRy3uvl0f5b1pg,14u0H4SMT951WtE1ika,dYQv7N3bRUcffoyZNJN,FUAV2F1PEN1UMpzb6re
later_30,93449.8,140.3,107.36,15140,309.45
later_60,93449.8,140.3,107.36,15140,309.45
duration,10.3333,22.6333,13.1667,20.7667,15.6333
min_date,2019-01-15,2018-01-11,2018-10-22,2018-03-08,2018-08-09
max_date,2019-11-19,2019-11-19,2019-11-12,2019-11-19,2019-11-19
ttl_amt,1.0398e+06,52142.3,77959.4,657930,86779.9
avg_mon_amt,100626,2303.79,5920.97,31682,5550.96
avg_ord_amt,2429.45,778.244,1025.78,1393.92,683.307
order_cnt,428,67,76,472,127


In [123]:
ss['cass_code'].value_counts()

C0146447    1
C0107518    1
C0125695    1
C0143198    1
C0115775    1
C0170688    1
C0130805    1
C0019566    1
C0102273    1
C0021102    1
C0130820    1
C0198597    1
C0183178    1
C0136908    1
C0111733    1
C0119356    1
C0180308    1
C0179840    1
C0105925    1
C0117394    1
C0132802    1
C0120583    1
C0123765    1
C0190778    1
C0193913    1
C0142351    1
C0018295    1
C0017976    1
C0165253    1
C0018299    1
           ..
C0189550    1
C0018540    1
C0148988    1
C0130919    1
C0199817    1
C0013713    1
C0193805    1
C0170384    1
C0168746    1
C0155273    1
C0185775    1
C0125235    1
C0199748    1
C0173922    1
C0146315    1
C0144578    1
C0153651    1
C0167706    1
C0146818    1
C0017725    1
C0195546    1
C0129093    1
C0198770    1
C0192855    1
C0189264    1
C0152947    1
C0148133    1
C0160879    1
C0122026    1
C0130196    1
Name: cass_code, Length: 273, dtype: int64

### 维表合并

In [11]:
data = pd.merge(pd.merge(sales,cust,left_on = 'company_id',right_on = 'id',how = 'left'),inquiry,left_on = 'company_id',right_on = 'garage_company_id',how = 'left')

In [12]:
data.head().T

,0,1,2,3,4
company_id,is0MU4l4Y0OsYPJGk5n,lpNCwtPRarHlfXuvrc1,12418,2KzewjZjjxdCyHLjxwr,p3cmI6GZwHph5OYqZzB
later_30,370.00000,6589.20000,201863.96000,3833.00000,16857.09000
later_60,370.00000,19929.20000,385372.23000,5750.20000,19409.99000
duration,6.33330,5.06670,22.40000,4.86670,3.83330
min_date,2019-04-29,2019-06-06,2018-01-02,2019-06-12,2019-07-13
max_date,2019-08-23,2019-11-03,2019-11-02,2019-10-27,2019-11-03
ttl_amt,33011.40000,47548.40000,5447146.79000,45058.00000,26000.77000
avg_mon_amt,5212.32632,9384.55263,243176.19598,9258.49315,6782.80956
avg_ord_amt,1500.51818,660.39444,1590.87231,1325.23529,962.99148
order_cnt,22.00000,72.00000,3424.00000,34.00000,27.00000


In [13]:
data.columns.tolist()

[u'company_id',
 u'later_30',
 u'later_60',
 u'duration',
 u'min_date',
 u'max_date',
 u'ttl_amt',
 u'avg_mon_amt',
 u'avg_ord_amt',
 u'order_cnt',
 u'avg_mon_cnt',
 u'min_amt',
 u'max_amt',
 u'mon_past_30',
 u'max_past_30',
 u'mon_past_60',
 u'max_past_60',
 u'mon_past_90',
 u'mon_past_120',
 u'mon_past_150',
 u'mon_past_180',
 u'mon_past_360',
 u'ord_past_30',
 u'ord_past_60',
 u'ord_past_90',
 u'ord_past_120',
 u'ord_past_150',
 u'ord_past_180',
 u'ord_past_360',
 u'id',
 u'name',
 u'code',
 u'group_classfication',
 u'auth_dur',
 u'crt_dur',
 u'province',
 u'is_enabled',
 u'is_virtual',
 u'is_authenticated',
 u'authenticated_stamp',
 u'created_stamp',
 u'province_geo_name',
 u'city_geo_name',
 u'county_geo_name',
 u'customer_manager',
 u'CASSLOAN_credit_limit',
 u'CASSLOAN_credit_limit_max',
 u'CASSLOAN_credit_st',
 u'CASSLOAN_MAX_DEAD_LINE',
 u'COMPANY_ACCOUNT_credit_limit',
 u'COMPANY_ACCOUNT_credit_st',
 u'COMPANY_ACCOUNT_credit_limit_max',
 u'COMPANY_ACCOUNT_MAX_DEAD_LINE',
 u'l

In [14]:
c = ['company_id',
'later_30',
'later_60',
'duration',
'ttl_amt',
'avg_mon_amt',
'avg_ord_amt',
'order_cnt',
'avg_mon_cnt',
'min_amt',
'max_amt',
'mon_past_30',
'max_past_30',
'mon_past_60',
'max_past_60',
'mon_past_90',
'mon_past_120',
'mon_past_150',
'mon_past_180',
'mon_past_360',
'ord_past_30',
'ord_past_60',
'ord_past_90',
'ord_past_120',
'ord_past_150',
'ord_past_180',
'ord_past_360',
'group_classfication',
'auth_dur',
'crt_dur',
'province',
'is_enabled',
'is_virtual',
'is_authenticated',
'luxury_past_all_inq_dur',
'luxury_past_all_inq_cnt',
'luxury_past_all_need_cnt',
'premium_past_all_inq_dur',
'premium_past_all_inq_cnt',
'premium_past_all_need_cnt',
'common_past_all_inq_dur',
'common_past_all_inq_cnt',
'common_past_all_need_cnt',
'cheap_past_all_inq_dur',
'cheap_past_all_inq_cnt',
'cheap_past_all_need_cnt',
'luxury_past_30_inq_dur',
'luxury_past_30_inq_cnt',
'luxury_past_30_need_cnt',
'premium_past_30_inq_dur',
'premium_past_30_inq_cnt',
'premium_past_30_need_cnt',
'common_past_30_inq_dur',
'common_past_30_inq_cnt',
'common_past_30_need_cnt',
'cheap_past_30_inq_dur',
'cheap_past_30_inq_cnt',
'cheap_past_30_need_cnt',
'luxury_past_60_inq_dur',
'luxury_past_60_inq_cnt',
'luxury_past_60_need_cnt',
'premium_past_60_inq_dur',
'premium_past_60_inq_cnt',
'premium_past_60_need_cnt',
'common_past_60_inq_dur',
'common_past_60_inq_cnt',
'common_past_60_need_cnt',
'cheap_past_60_inq_dur',
'cheap_past_60_inq_cnt',
'cheap_past_60_need_cnt']

clean = data[c]
clean.head().T

,0,1,2,3,4
company_id,is0MU4l4Y0OsYPJGk5n,lpNCwtPRarHlfXuvrc1,12418,2KzewjZjjxdCyHLjxwr,p3cmI6GZwHph5OYqZzB
later_30,370.00000,6589.20000,201863.96000,3833.00000,16857.09000
later_60,370.00000,19929.20000,385372.23000,5750.20000,19409.99000
duration,6.33330,5.06670,22.40000,4.86670,3.83330
ttl_amt,33011.40000,47548.40000,5447146.79000,45058.00000,26000.77000
avg_mon_amt,5212.32632,9384.55263,243176.19598,9258.49315,6782.80956
avg_ord_amt,1500.51818,660.39444,1590.87231,1325.23529,962.99148
order_cnt,22.00000,72.00000,3424.00000,34.00000,27.00000
avg_mon_cnt,3.47370,14.21050,152.85710,6.98630,7.04350
min_amt,100.00000,15.00000,8.00000,30.00000,85.60000


## 2. 特征工程

In [15]:
# 各特征的缺失值数量，全是数值型特征，可以用0替代NaN
aa = clean.isnull().sum()
aa[aa>0].sort_values(ascending=False)

ord_past_30                  6896
ord_past_60                  5478
ord_past_90                  4904
ord_past_120                 4598
ord_past_150                 4351
ord_past_180                 4178
ord_past_360                 3916
max_amt                      3790
duration                     3790
min_amt                      3790
avg_mon_cnt                  3790
order_cnt                    3790
avg_ord_amt                  3790
avg_mon_amt                  3790
ttl_amt                      3790
cheap_past_all_inq_dur       2750
luxury_past_all_inq_dur      2750
luxury_past_all_inq_cnt      2750
luxury_past_all_need_cnt     2750
premium_past_all_inq_dur     2750
premium_past_all_inq_cnt     2750
premium_past_all_need_cnt    2750
common_past_all_inq_dur      2750
common_past_all_inq_cnt      2750
common_past_all_need_cnt     2750
cheap_past_60_need_cnt       2750
cheap_past_60_inq_cnt        2750
cheap_past_30_need_cnt       2750
cheap_past_60_inq_dur        2750
common_past_60

In [16]:
clean = clean.fillna(0)
# 再查看各特征的缺失值数量
aa = clean.isnull().sum()
aa[aa>0].sort_values(ascending=False)
# 缺失值全部补上

Series([], dtype: int64)

In [17]:
# 离散变量赋值
for c in ['group_classfication','province','is_enabled','is_virtual','is_authenticated']:
    clean = clean.join(pd.get_dummies(clean[c],prefix = c))
    clean = clean.drop(c,1)

# 删除只有一类的特征
clean = clean.drop(['group_classfication_COMPANY','is_virtual_0','is_authenticated_1'], 1)

clean.head().T

,0,1,2,3,4
company_id,is0MU4l4Y0OsYPJGk5n,lpNCwtPRarHlfXuvrc1,12418,2KzewjZjjxdCyHLjxwr,p3cmI6GZwHph5OYqZzB
later_30,370.00000,6589.20000,201863.96000,3833.00000,16857.09000
later_60,370.00000,19929.20000,385372.23000,5750.20000,19409.99000
duration,6.33330,5.06670,22.40000,4.86670,3.83330
ttl_amt,33011.40000,47548.40000,5447146.79000,45058.00000,26000.77000
avg_mon_amt,5212.32632,9384.55263,243176.19598,9258.49315,6782.80956
avg_ord_amt,1500.51818,660.39444,1590.87231,1325.23529,962.99148
order_cnt,22.00000,72.00000,3424.00000,34.00000,27.00000
avg_mon_cnt,3.47370,14.21050,152.85710,6.98630,7.04350
min_amt,100.00000,15.00000,8.00000,30.00000,85.60000


In [18]:
clean.loc[clean.province_other == 1] 

,company_id,later_30,later_60,duration,ttl_amt,avg_mon_amt,avg_ord_amt,order_cnt,avg_mon_cnt,min_amt,max_amt,mon_past_30,max_past_30,mon_past_60,max_past_60,mon_past_90,mon_past_120,mon_past_150,mon_past_180,mon_past_360,ord_past_30,ord_past_60,ord_past_90,ord_past_120,ord_past_150,ord_past_180,ord_past_360,auth_dur,crt_dur,luxury_past_all_inq_dur,luxury_past_all_inq_cnt,luxury_past_all_need_cnt,premium_past_all_inq_dur,premium_past_all_inq_cnt,premium_past_all_need_cnt,common_past_all_inq_dur,common_past_all_inq_cnt,common_past_all_need_cnt,cheap_past_all_inq_dur,cheap_past_all_inq_cnt,cheap_past_all_need_cnt,luxury_past_30_inq_dur,luxury_past_30_inq_cnt,luxury_past_30_need_cnt,premium_past_30_inq_dur,premium_past_30_inq_cnt,premium_past_30_need_cnt,common_past_30_inq_dur,common_past_30_inq_cnt,common_past_30_need_cnt,cheap_past_30_inq_dur,cheap_past_30_inq_cnt,cheap_past_30_need_cnt,luxury_past_60_inq_dur,luxury_past_60_inq_cnt,luxury_past_60_need_cnt,premium_past_60_inq_dur,premium_past_60_inq_cnt,premium_past_60_need_cnt,common_past_60_inq_dur,common_past_60_inq_cnt,common_past_60_need_cnt,cheap_past_60_inq_dur,cheap_past_60_inq_cnt,cheap_past_60_need_cnt,province_other,province_province1,province_province2,province_province3,province_province4,is_enabled_0,is_enabled_1
38,C2rw1LuldvuWx0Zi9pC,4835.84000,4835.84000,6.33330,6573.11000,1037.85947,1314.62200,5.00000,0.78950,399.00000,2247.47000,0.00000,0.00000,693.32000,1386.64000,1908.03667,1431.02750,1144.82200,1020.51833,547.75917,0.00000,1386.64000,1908.03667,1908.03667,1908.03667,1530.77750,1314.62200,189.00000,190,0.00000,0.00000,0.00000,189.00000,32.00000,41.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,30.00000,2.00000,2.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,60.00000,4.00000,4.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1,0,0,0,0,0,1
1762,ym1GKBBeUeUveLWeUEa,2070.58000,2604.18000,5.80000,35511.48000,6122.66897,1076.10545,33.00000,5.68970,40.00000,8720.00000,5954.15000,2402.15000,8322.47500,8720.00000,6303.28333,5589.94500,5118.01600,5918.58000,2959.29000,992.35833,1849.43889,1350.70357,1315.28118,1279.50400,1076.10545,1076.10545,174.00000,175,133.00000,26.00000,39.00000,170.00000,336.00000,644.00000,1.00000,2.00000,2.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,30.00000,64.00000,120.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,60.00000,8.00000,9.00000,60.00000,108.00000,249.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1,0,0,0,0,0,1
2744,506z4QY5a6DsNg6xarF,171848.00000,217935.00000,10.20000,314416.00000,30825.09804,7486.09524,42.00000,4.11760,160.00000,45996.00000,20200.00000,20200.00000,32236.00000,23584.00000,24242.33333,34410.50000,28080.00000,28385.00000,26201.33333,20200.00000,10745.33333,9090.87500,8602.62500,7020.00000,7096.25000,7486.09524,496.00000,507,148.00000,4.00000,5.00000,134.00000,10.00000,34.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1,0,0,0,0,0,1
10101,0pr6lte2pyQj9Wj5GX6,4045.67000,4891.75000,7.33330,9263.32000,1263.18000,661.66571,14.00000,1.90910,25.98000,2891.35000,0.00000,0.00000,0.00000,0.00000,1067.30333,1078.57750,950.46200,1348.54500,771.94333,0.00000,0.00000,1600.95500,1078.57750,792.05167,899.03000,661.66571,220.00000,220,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1,0,0,0,0,0,1
10766,14422,1505.34000,2914.26000,19.23330,164975.15000,8577.56413,4999.24697,33.00000,1.71580,87.00000,35560.00000,0.00000,0.00000,7750.85000,14696.20000,5522.39333,4644.40000,6604.52800,8098.78667,8245.03500,0.00000,77

In [19]:
cust.loc[cust.id == 'hmToY3kVQdi8dZbpA9M']

,id,name,code,group_classfication,auth_dur,crt_dur,province,is_enabled,is_virtual,is_authenticated,authenticated_stamp,created_stamp,province_geo_name,city_geo_name,county_geo_name,customer_manager,CASSLOAN_credit_limit,CASSLOAN_credit_limit_max,CASSLOAN_credit_st,CASSLOAN_MAX_DEAD_LINE,COMPANY_ACCOUNT_credit_limit,COMPANY_ACCOUNT_credit_st,COMPANY_ACCOUNT_credit_limit_max,COMPANY_ACCOUNT_MAX_DEAD_LINE,legal_person_identity,business_license_name


In [26]:
clean.head().T

,0,1,2,3,4
company_id,0vnU2gfKgjQznjz8rna,wsSMjhLU3A84yhdVRCA,12101,kqCb8FNOufck0ruXimY,12740
later_30,65668.93000,11657.00000,56114.40000,552782.00000,20765.30000
later_60,65668.93000,11657.00000,56114.40000,552782.00000,20765.30000
duration,12.03330,11.46670,22.43330,9.66670,22.50000
ttl_amt,723624.59000,70383.74000,1731290.91000,10863383.58000,2189407.50000
avg_mon_amt,60135.00748,6138.11686,77174.92912,1123798.30138,97307.00000
avg_ord_amt,888.97370,3060.16261,1082.73353,9571.26307,1608.67561
order_cnt,814.00000,23.00000,1599.00000,1135.00000,1361.00000
avg_mon_cnt,67.64540,2.00580,71.27790,117.41380,60.48890
min_amt,5.00000,107.02000,10.00000,50.00000,10.00000


In [20]:
df = clean.iloc[:, 1:]
df

,later_30,later_60,duration,ttl_amt,avg_mon_amt,avg_ord_amt,order_cnt,avg_mon_cnt,min_amt,max_amt,mon_past_30,max_past_30,mon_past_60,max_past_60,mon_past_90,mon_past_120,mon_past_150,mon_past_180,mon_past_360,ord_past_30,ord_past_60,ord_past_90,ord_past_120,ord_past_150,ord_past_180,ord_past_360,auth_dur,crt_dur,luxury_past_all_inq_dur,luxury_past_all_inq_cnt,luxury_past_all_need_cnt,premium_past_all_inq_dur,premium_past_all_inq_cnt,premium_past_all_need_cnt,common_past_all_inq_dur,common_past_all_inq_cnt,common_past_all_need_cnt,cheap_past_all_inq_dur,cheap_past_all_inq_cnt,cheap_past_all_need_cnt,luxury_past_30_inq_dur,luxury_past_30_inq_cnt,luxury_past_30_need_cnt,premium_past_30_inq_dur,premium_past_30_inq_cnt,premium_past_30_need_cnt,common_past_30_inq_dur,common_past_30_inq_cnt,common_past_30_need_cnt,cheap_past_30_inq_dur,cheap_past_30_inq_cnt,cheap_past_30_need_cnt,luxury_past_60_inq_dur,luxury_past_60_inq_cnt,luxury_past_60_need_cnt,premium_past_60_inq_dur,premium_past_60_inq_cnt,premium_past_60_need_cnt,common_past_60_inq_dur,common_past_60_inq_cnt,common_past_60_need_cnt,cheap_past_60_inq_dur,cheap_past_60_inq_cnt,cheap_past_60_need_cnt,province_other,province_province1,province_province2,province_province3,province_province4,is_enabled_0,is_enabled_1
0,370.00000,370.00000,6.33330,33011.40000,5212.32632,1500.51818,22.00000,3.47370,100.00000,8299.20000,0.00000,0.00000,0.00000,0.00000,220.00000,908.25000,1541.00000,1335.83333,2750.95000,0.00000,0.00000,330.00000,454.12500,513.66667,471.47059,1500.51818,192.00000,192,82.00000,6.00000,14.00000,174.00000,56.00000,164.00000,182.00000,70.00000,126.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,30.00000,4.00000,10.00000,30.00000,8.00000,20.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,60.00000,6.00000,15.00000,60.00000,16.00000,36.00000,0.00000,0.00000,0.00000,0,0,1,0,0,0,1
1,6589.20000,19929.20000,5.06670,47548.40000,9384.55263,660.39444,72.00000,14.21050,15.00000,11430.00000,7200.19000,1466.19000,6221.77000,1466.19000,7366.11333,10987.80000,9422.48000,7924.73333,3962.36667,600.01583,414.78467,450.98653,686.73750,663.55493,660.39444,660.39444,313.00000,528,0.00000,0.00000,0.00000,309.00000,104.00000,281.00000,313.00000,354.00000,804.00000,36.00000,4.00000,10.00000,0.00000,0.00000,0.00000,30.00000,20.00000,44.00000,30.00000,58.00000,130.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,60.00000,40.00000,121.00000,60.00000,136.00000,301.00000,0.00000,0.00000,0.00000,0,0,0,0,1,0,1
2,201863.96000,385372.23000,22.40000,5447146.79000,243176.19598,1590.87231,3424.00000,152.85710,8.00000,85000.00000,140146.37000,11007.00000,168398.04500,20330.00000,192599.91333,207541.43750,209150.06800,195874.87167,213925.17083,881.42371,1005.36146,1077.98459,1109.84726,1147.91475,1106.63769,1277.16520,943.00000,1069,669.00000,1074.00000,3519.00000,672.00000,6422.00000,17294.00000,666.00000,948.00000,2702.00000,0.00000,0.00000,0.00000,30.00000,36.00000,115.00000,30.00000,290.00000,741.00000,30.00000,110.00000,313.00000,0.00000,0.00000,0.00000,60.00000,92.00000,364.00000,60.00000,529.00000,1522.00000,60.00000,210.00000,685.00000,0.00000,0.00000,0.00000,0,0,1,0,0,0,1
3,3833.00000,5750.20000,4.86670,45058.00000,9258.49315,1325.23529,34.00000,6.98630,30.00000,4225.00000,12730.00000,3800.00000,10707.50000,3800.00000,11719.33333,11170.75000,9011.60000,7509.66667,3754.83333,1414.44444,1259.70588,1171.93333,1354.03030,1325.23529,1325.23529,1325.23529,173.00000,173,11.00000,6.00000,8.00000,166.00000,140.00000,345.00000,158.00000,30.00000,40.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,30.00000,34.00000,52.00000,30.00000,8.00000,10.00000,0.00000,0.00000,0.00000,60.00000,6.00000,8.00000,60.00000,58.00000,127.00000,60.00000,12.00000,16.00000,0.00000,0.00000,0.00000,0,0,1,0,0,0,1
4,16857.09000,19409.99000,3.83330,26000.77000,6782.80956,962.99148,27.00000,7.04350,85.60000,5056.20000,1752.65000,795.00000,4237.95000,5056.20000,7040.38667,6500.19250,5200.15400,4333.46167,2166.73083,43

In [21]:
len(df[df.later_30 <> df.later_60])

15452

In [22]:
X = df.iloc[:, 2:]
Y = df.later_60

In [23]:
X.head()

,duration,ttl_amt,avg_mon_amt,avg_ord_amt,order_cnt,avg_mon_cnt,min_amt,max_amt,mon_past_30,max_past_30,mon_past_60,max_past_60,mon_past_90,mon_past_120,mon_past_150,mon_past_180,mon_past_360,ord_past_30,ord_past_60,ord_past_90,ord_past_120,ord_past_150,ord_past_180,ord_past_360,auth_dur,crt_dur,luxury_past_all_inq_dur,luxury_past_all_inq_cnt,luxury_past_all_need_cnt,premium_past_all_inq_dur,premium_past_all_inq_cnt,premium_past_all_need_cnt,common_past_all_inq_dur,common_past_all_inq_cnt,common_past_all_need_cnt,cheap_past_all_inq_dur,cheap_past_all_inq_cnt,cheap_past_all_need_cnt,luxury_past_30_inq_dur,luxury_past_30_inq_cnt,luxury_past_30_need_cnt,premium_past_30_inq_dur,premium_past_30_inq_cnt,premium_past_30_need_cnt,common_past_30_inq_dur,common_past_30_inq_cnt,common_past_30_need_cnt,cheap_past_30_inq_dur,cheap_past_30_inq_cnt,cheap_past_30_need_cnt,luxury_past_60_inq_dur,luxury_past_60_inq_cnt,luxury_past_60_need_cnt,premium_past_60_inq_dur,premium_past_60_inq_cnt,premium_past_60_need_cnt,common_past_60_inq_dur,common_past_60_inq_cnt,common_past_60_need_cnt,cheap_past_60_inq_dur,cheap_past_60_inq_cnt,cheap_past_60_need_cnt,province_other,province_province1,province_province2,province_province3,province_province4,is_enabled_0,is_enabled_1
0,6.33330,33011.40000,5212.32632,1500.51818,22.00000,3.47370,100.00000,8299.20000,0.00000,0.00000,0.00000,0.00000,220.00000,908.25000,1541.00000,1335.83333,2750.95000,0.00000,0.00000,330.00000,454.12500,513.66667,471.47059,1500.51818,192.00000,192,82.00000,6.00000,14.00000,174.00000,56.00000,164.00000,182.00000,70.00000,126.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,30.00000,4.00000,10.00000,30.00000,8.00000,20.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,60.00000,6.00000,15.00000,60.00000,16.00000,36.00000,0.00000,0.00000,0.00000,0,0,1,0,0,0,1
1,5.06670,47548.40000,9384.55263,660.39444,72.00000,14.21050,15.00000,11430.00000,7200.19000,1466.19000,6221.77000,1466.19000,7366.11333,10987.80000,9422.48000,7924.73333,3962.36667,600.01583,414.78467,450.98653,686.73750,663.55493,660.39444,660.39444,313.00000,528,0.00000,0.00000,0.00000,309.00000,104.00000,281.00000,313.00000,354.00000,804.00000,36.00000,4.00000,10.00000,0.00000,0.00000,0.00000,30.00000,20.00000,44.00000,30.00000,58.00000,130.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,60.00000,40.00000,121.00000,60.00000,136.00000,301.00000,0.00000,0.00000,0.00000,0,0,0,0,1,0,1
2,22.40000,5447146.79000,243176.19598,1590.87231,3424.00000,152.85710,8.00000,85000.00000,140146.37000,11007.00000,168398.04500,20330.00000,192599.91333,207541.43750,209150.06800,195874.87167,213925.17083,881.42371,1005.36146,1077.98459,1109.84726,1147.91475,1106.63769,1277.16520,943.00000,1069,669.00000,1074.00000,3519.00000,672.00000,6422.00000,17294.00000,666.00000,948.00000,2702.00000,0.00000,0.00000,0.00000,30.00000,36.00000,115.00000,30.00000,290.00000,741.00000,30.00000,110.00000,313.00000,0.00000,0.00000,0.00000,60.00000,92.00000,364.00000,60.00000,529.00000,1522.00000,60.00000,210.00000,685.00000,0.00000,0.00000,0.00000,0,0,1,0,0,0,1
3,4.86670,45058.00000,9258.49315,1325.23529,34.00000,6.98630,30.00000,4225.00000,12730.00000,3800.00000,10707.50000,3800.00000,11719.33333,11170.75000,9011.60000,7509.66667,3754.83333,1414.44444,1259.70588,1171.93333,1354.03030,1325.23529,1325.23529,1325.23529,173.00000,173,11.00000,6.00000,8.00000,166.00000,140.00000,345.00000,158.00000,30.00000,40.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,30.00000,34.00000,52.00000,30.00000,8.00000,10.00000,0.00000,0.00000,0.00000,60.00000,6.00000,8.00000,60.00000,58.00000,127.00000,60.00000,12.00000,16.00000,0.00000,0.00000,0.00000,0,0,1,0,0,0,1
4,3.83330,26000.77000,6782.80956,962.99148,27.00000,7.04350,85.60000,5056.20000,1752.65000,795.00000,4237.95000,5056.20000,7040.38667,6500.19250,5200.15400,4333.46167,2166.73083,438.16250,847.59000,1111.64000,962.99148,962.99148,962.99148,962.99148,200.00000,200,0.00000,0.00000,0.00000,171.00000,30.00000,131.000

In [24]:
Y.head()

0      370.00000
1    19929.20000
2   385372.23000
3     5750.20000
4    19409.99000
Name: later_60, dtype: float64

In [25]:
import sklearn as sk
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=0)

In [26]:
print len(X_train)
print len(y_train)
print len(X_test)
print len(y_test)

14463
14463
3616
3616


In [27]:
# define cross validation strategy
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse

In [28]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor

In [29]:
models = [LinearRegression(),Ridge(),Lasso(alpha=0.01,max_iter=10000),RandomForestRegressor(),GradientBoostingRegressor(),SVR(),LinearSVR(),
          ElasticNet(alpha=0.001,max_iter=10000),SGDRegressor(max_iter=1000,tol=1e-3),BayesianRidge(),KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5),
          ExtraTreesRegressor(),XGBRegressor()]

In [31]:
names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra","Xgb"]
for name, model in zip(names, models):
    score = rmse_cv(model, X_train, y_train)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

LR: 87654.119988, 16681.7953


d:\ProgramData\Anaconda2\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=6.81053e-17): result may not be accurate.
  overwrite_a=True).T
d:\ProgramData\Anaconda2\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=7.00196e-17): result may not be accurate.
  overwrite_a=True).T
d:\ProgramData\Anaconda2\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=6.65079e-17): result may not be accurate.
  overwrite_a=True).T
d:\ProgramData\Anaconda2\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=9.83354e-17): result may not be accurate.
  overwrite_a=True).T


Ridge: 87620.617012, 16655.3383


d:\ProgramData\Anaconda2\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Lasso: 87653.935952, 16681.7360


...

LinSVR: 184790.384515, 107313.4467
Ela: 87501.225842, 16544.0266
SGD: 363210706799824601088.000000, 374683022589845831680.0000
Bay: 84876.639132, 13449.2471


d:\ProgramData\Anaconda2\lib\site-packages\sklearn\linear_model\ridge.py:167: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


Ker: 1623015.820629, 1788049.3620
Extra: 80987.679436, 8456.5299


d:\ProgramData\Anaconda2\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Xgb: 74785.062559, 9210.2209


In [41]:
# 第一轮模型筛选后，随机森林看起来效果最好，接下去对random forest进行调参
rf = RandomForestRegressor(n_estimators= 1000, random_state=42)
rf.fit(X_train, y_train)
print  1- sum(abs(rf.predict(X_train) - y_train))/sum(y_train)
print  1- sum(abs(rf.predict(X_test) - y_test))/sum(y_test)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [49]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
print  1- sum(abs(rf.predict(X_train) - y_train))/sum(y_train)
print  1- sum(abs(rf.predict(X_test) - y_test))/sum(y_test)


0.742741836389
0.31070955068


In [58]:
importances = list(rf.feature_importances_)
# feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# feature_importances

[0.003963544413570431,
 0.07362474850328517,
 0.23277362199840548,
 0.0008203198274298022,
 0.0014716188531066455,
 0.0027014741079221783,
 0.00947030390528573,
 0.0031781601014539544,
 0.031036589552195147,
 0.002164822169134247,
 0.029721034409098523,
 0.003001168924897317,
 0.08278491489516641,
 0.02318445451550907,
 0.024333520671531435,
 0.034639337707413106,
 0.07456064313777783,
 0.002596987571206418,
 0.08115961249983544,
 0.0021354026787392517,
 0.0005729251627835211,
 0.06471251184553804,
 0.0021139277604002495,
 0.13174649617155887,
 0.0018158947206042165,
 0.0029903433544512985,
 0.0009278334826025288,
 0.0026971104750978516,
 0.0004967910219143618,
 0.002349661593586818,
 0.003049843578064464,
 0.0017928053962068587,
 0.0006927537646090394,
 0.0008796797282851796,
 0.0011747952567686868,
 0.00017567338570207748,
 0.00013957758277947913,
 0.00012876233530162475,
 2.4795131989610782e-05,
 0.000755858347096718,
 0.0005805791409364908,
 9.515571010159312e-05,
 0.00255591534173